In [17]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key=os.getenv('groq_api')
travly=os.getenv('travly')
os.environ["TAVILY_API_KEY"] = travly


from typing import Annotated,TypedDict,Sequence

from langchain_core.messages import BaseMessage,SystemMessage,HumanMessage
from langgraph.graph.message import add_messages
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph,END,START
# The tools our agent can use
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode



In [42]:
# The state for our agent
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

@tool
def add(a:int,b:int):
   """This is an addition function that adds 2 numbers"""
   return a+b
@tool
def subtract(a:int,b:int):
    """This is a subtraction function that adds 2 numbers"""
    return a-b
@tool
def multiply(a:int,b:int):
    """This is a Multiply function that adds 2 numbers"""
    return a*b
tools=[add,subtract,multiply]
llm = ChatGroq(api_key=api_key, model="Gemma2-9b-It").bind_tools(tools)

In [43]:
def model_call(state:AgentState):
    system_message=SystemMessage(content='You are my ai assistant, please anser my query to the best of your ability')
    response=llm.invoke([system_message]+state['messages'])
    return {'messages':response}

In [44]:
def should_cont(state:AgentState):
    last_message=state['messages'][-1]
    if not last_message.tool_calls:
        return "end"
    else:
        return "continue"
    



In [45]:
graph=StateGraph(AgentState)

graph.add_node("our_agent",model_call)

tool_node=ToolNode(tools=tools)
graph.add_node('tools',tool_node)

graph.add_edge(START,'our_agent')
graph.add_conditional_edges(
    "our_agent",
    should_cont,
    {
        "continue":"tools",
        "end":END
    },
)
graph.add_edge('tools','our_agent')

app=graph.compile()

In [51]:

app.invoke({'messages':HumanMessage('add 3 and 4 then multiply by 3 then subtract 4,then give me a joke')})


{'messages': [HumanMessage(content='add 3 and 4 then multiply by 3 then subtract 4,then give me a joke', additional_kwargs={}, response_metadata={}, id='a0d62ee9-2c26-4ef3-b7a0-523130d23cb1'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'n880kn2zv', 'function': {'arguments': '{"a":3,"b":4}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 1254, 'total_tokens': 1342, 'completion_time': 0.16, 'prompt_time': 0.023674493, 'queue_time': 0.253436147, 'total_time': 0.183674493}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--0f90b6b8-b6b2-4260-aa6a-93076dbfdf57-0', tool_calls=[{'name': 'add', 'args': {'a': 3, 'b': 4}, 'id': 'n880kn2zv', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1254, 'output_tokens': 88, 'total_tokens': 1342}),
  ToolMessage(content='7', name='add', id='eb790979-a6ed-